In [ ]:
import cv2
import imutils
import dlib
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
glasses = cv2.imread("./data/glass_template.png", -1)
glasses = glasses[150:800,:]

plt.imshow(cv2.cvtColor(glasses,cv2.COLOR_RGBA2RGB))

In [ ]:
no_glasses_array = np.load('./data//no_glasses.npy')

fig = plt.figure(figsize=[3,3])
plt.imshow(no_glasses_array[0])

In [ ]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./models/shape_predictor_68_face_landmarks.dat")

In [ ]:
def image_subtract(a, b):
    """Subtract two uint8 arrays without overflow."""
    # Compute the subtraction in a larger data type (e.g., int16)
    result = a.astype(np.int16) - b.astype(np.int16)
    # Clip the result to the range [0, 255]
    result = np.clip(result, 0, 255)
    # Convert the result back to uint8
    return result.astype(np.uint8)

In [ ]:
def apply_glass_template(image):
    
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    dets = detector(gray, 1)

    #find face box bounding points
    for d in dets:

        x = d.left()
        y = d.top()
        w = d.right()
        h = d.bottom()

    dlib_rect = dlib.rectangle(x, y, w, h)

    ##############   Find facial landmarks   ##############
    detected_landmarks = predictor(gray, dlib_rect).parts()

    landmarks = np.matrix([[p.x, p.y] for p in detected_landmarks])

    for idx, point in enumerate(landmarks):
        pos = (point[0, 0], point[0, 1])
        if idx == 0:
            eye_left = pos
        elif idx == 16:
            eye_right = pos

    eye_center = (eye_left[1] + eye_right[1]) / 2

    #glasses translation
    glass_trans = int(.4 * (eye_center - y))

    # resize glasses to width of face and blend images
    if w > 112:
        w = 112
    if h > 112:
        h = 112

    face_width = w - x

    # resize_glasses
    glasses_resize = imutils.resize(glasses, face_width)

    yG, xG, cG = glasses_resize.shape
    glasses_resize = cv2.cvtColor(255 - cv2.cvtColor(glasses_resize, cv2.COLOR_RGBA2GRAY),cv2.COLOR_GRAY2RGB)

    image_fit = image[y + glass_trans:y + yG + glass_trans, x:w]
    
    blend = image_subtract(image_fit,glasses_resize)

    h5, w5, s5 = image_fit.shape
    image[y + glass_trans:y + h5 + glass_trans, x:x+w5 ] = blend

    return image


In [ ]:
result_array = np.zeros_like(no_glasses_array)

for i in range(no_glasses_array.shape[0]):
    result_array[i] = apply_glass_template(no_glasses_array[i])

In [ ]:
np.save('./data//template_glasses.npy',result_array)